<a href="https://colab.research.google.com/github/Jeorr/ML/blob/main/Lab1PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Встановлення бібліотек
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Завантаження даних
url = "https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/refs/heads/master/insurance.csv"
data = pd.read_csv(url)

# Перетворення номінативних даних на числові за допомогою LabelEncoder
label_encoder = LabelEncoder()
data['sex'] = label_encoder.fit_transform(data['sex'])
data['smoker'] = label_encoder.fit_transform(data['smoker'])
data['region'] = label_encoder.fit_transform(data['region'])

# Визначення вхідних та цільових змінних
X = data.drop(['charges'], axis=1).values  # Всі змінні, крім 'charges', є вхідними
y = data['charges'].values  # 'charges' — цільова змінна

# Розділення на тренувальні та тестові набори (80% на тренування, 20% на тестування)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормалізація даних
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Перетворення даних у тензори PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Зміна форми для виходу
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Створення моделі нейронної мережі
class InsuranceModel(nn.Module):
    def __init__(self):
        super(InsuranceModel, self).__init__()
        self.fc1 = nn.Linear(6, 256)  # Вхідний шар
        self.fc2 = nn.Linear(256, 128)  # Другий шар
        self.fc3 = nn.Linear(128, 64)  # Третій шар
        self.fc4 = nn.Linear(64, 1)  # Вихідний шар

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = nn.Dropout(0.2)(x)  # Додаємо Dropout, без параметра training
        x = torch.relu(self.fc2(x))
        x = nn.Dropout(0.2)(x)  # Додаємо Dropout, без параметра training
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)  # Вихід
        return x

# Ініціалізація моделі
model = InsuranceModel()

# Визначення втрат і оптимізатора
criterion = nn.MSELoss()  # Використання середньоквадратичної помилки
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Навчання моделі
epochs = 150
for epoch in range(epochs):
    model.train()  # Переведення моделі в режим навчання
    optimizer.zero_grad()  # Обнулення градієнтів
    y_pred = model(X_train_tensor)  # Прогнозування
    loss = criterion(y_pred, y_train_tensor)  # Обчислення втрат
    loss.backward()  # Обчислення градієнтів
    optimizer.step()  # Оновлення ваг моделі

    # Вивід втрат на кожній 10-й епосі
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Прогнозування на тестовій вибірці
model.eval()  # Переведення моделі в режим оцінки
with torch.no_grad():  # Без обчислення градієнтів
    y_test_pred = model(X_test_tensor)

# Обчислення метрик MAE та MSE
mae = mean_absolute_error(y_test_tensor.numpy(), y_test_pred.numpy())
mse = mean_squared_error(y_test_tensor.numpy(), y_test_pred.numpy())

# Виведення результатів
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')

# Перевірка, чи MAE є прийнятним
acceptable_mae = np.mean(y) * 0.1  # 10% від середньої вартості

if mae <= acceptable_mae:
    print(f"MAE = {mae} є прийнятним.")
else:
    print(f"MAE = {mae} є занадто великим.")

# Додатково, ви можете порівняти MSE з квадратом середнього
mse_comparison = np.mean(y) ** 2

if mse <= mse_comparison:
    print(f"MSE = {mse} є прийнятним в порівнянні з квадратом середнього.")
else:
    print(f"MSE = {mse} є занадто великим.")


Epoch [10/150], Loss: 322406080.0000
Epoch [20/150], Loss: 322242336.0000
Epoch [30/150], Loss: 321807488.0000
Epoch [40/150], Loss: 320846816.0000
Epoch [50/150], Loss: 318976448.0000
Epoch [60/150], Loss: 315601408.0000
Epoch [70/150], Loss: 309923040.0000
Epoch [80/150], Loss: 301071936.0000
Epoch [90/150], Loss: 288078848.0000
Epoch [100/150], Loss: 269266304.0000
Epoch [110/150], Loss: 244908400.0000
Epoch [120/150], Loss: 213621776.0000
Epoch [130/150], Loss: 178289248.0000
Epoch [140/150], Loss: 142153472.0000
Epoch [150/150], Loss: 108326216.0000
Mean Absolute Error (MAE): 6554.75732421875
Mean Squared Error (MSE): 104056680.0
MAE = 6554.75732421875 є занадто великим.
MSE = 104056680.0 є прийнятним в порівнянні з квадратом середнього.
